<h2>Baysian Recommender</h2>
<p>In this notebook I calculate all the posterior distributions for all the goalies playing at that time and match those similar to the selected goalie with the thought that it could be an additional feature for the dashboard</p>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime as dt, timedelta
from dateutil.relativedelta import *
import re
import sqlite3
import scipy.stats as stats
pd.set_option('display.max_rows', 500)

<h3>SQLite Handling functions</h3>
<p>Use Pandas to import dataframe into SQLlite</p>

In [3]:
DB = "..\\Data\\hockey_data_goalies.db"

#SQL Handle functions
def run_query(q):
    with sqlite3.connect(DB) as conn:
        x = pd.read_sql(q,conn)
    return x
   
#send command
def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.isolation_level = None
        return conn.execute(c)
    
#show tables
def show_tables():
    r = """
    Select name,type
    FROM sqlite_master
    WHERE type in ("table","view");
        """
    return run_query(r)

<h2>Bayesian Calculator</h2>
    <p>For a given data (in this case, Jan 17 2017), calculate the postierior mean and variance for all the active goalies in the database.</p>

In [4]:
a = 231.14
b = 24.2
q = """SELECT player_id,SUM(saves) saves, SUM(goals_against) goals
        FROM player_log log
        LEFT JOIN player_list pl ON pl.unique_id= log.player_id
        WHERE (CAST(SUBSTR(date_game,1,4) AS FLOAT)+CAST(SUBSTR(date_game,6,7) AS FLOAT)/12) > {0}
        AND date_game < \"{1}\"
        GROUP BY player_id
        """.format(2016.66,'2017-01-17')
active_players = run_query(q)
active_players['alpha_prime'] = active_players['saves'] + a
active_players['beta_prime'] = active_players['goals'] + b
active_players['mean'] = active_players['alpha_prime']/(active_players['alpha_prime'] +active_players['beta_prime'])
active_players['var'] = np.sqrt((active_players['alpha_prime']*active_players['beta_prime'])/((active_players['alpha_prime'] +active_players['beta_prime'])**2+(1+active_players['alpha_prime'] +active_players['beta_prime'])))

<h2>Pick a Similar Goalies</h2>
<p>For a random goalie, pick three similar goalies based on mean and varience by minimising the normalised RMSE.</p>

In [15]:
#pick a random goalie
player = active_players.iloc[np.random.randint(0,active_players.shape[0])]
#calculate current save percentage
active_players['save_pct']= active_players['saves']/(active_players['goals']+active_players['saves'])

In [16]:
#difference
active_players['distance'] = np.sqrt(((active_players['mean']-player['mean'])/player['mean'])**2+((active_players['var']-player['var'])/player['var'])**2)
#get top three recommendations
active_players.sort_values('distance')[1:4]

,player_id,saves,goals,alpha_prime,beta_prime,mean,var,distance,save_pct
26,hammoan01,47,8,278.14,32.2,0.896243,0.304454,0.000145,0.854545
50,mrazepe01,602,72,833.14,96.2,0.896486,0.304466,0.000171,0.893175
73,zatkoje01,178,23,409.14,47.2,0.896568,0.304188,0.001000,0.885572
